Message to Grader: 

This code doesn't work! Please don't try to run it. This ipnyb is something I wrote because my current implementation in the context of the Spot Mini Mini repository does not work properly and I knew you needed something readable to grade. The majority of the code located in those files are NOT mine, but there was a major change I had to fix in order to get the simulation working. 

This file is mainly to explain the concept, provide code snippets, and I will try my best to explain what went wrong, why it went wrong (if I understand correctly), and how I would fix the implementation in the future. 

Thank you for your understanding!

Best, 
Angelina

This first section describes the changes I made to the repository files in order to enable the simulation to properly run. I'm not sure entirely how it broke, but the repository is close to 5 years old, so I figured it was bound to have issues somewhere. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import argparse
import sys
import pickle

sys.path.append('../../')

from spotmicro.util.gui import GUI
from spotmicro.GymEnvs.spot_bezier_env import SpotBezierEnv
from spotmicro.Kinematics.SpotKinematics import SpotModel
from spotmicro.GaitGenerator.Bezier import BezierGait
from spotmicro.OpenLoopSM.SpotOL import BezierStepper
from spotmicro.spot_env_randomizer import SpotEnvRandomizer
from ars_lib.ars import ARSAgent, Normalizer, Policy, ParallelWorker

import multiprocessing as mp
from multiprocessing import Pipe


In [ ]:
# spot_ars.py line 135

# I also decreased the max_timesteps and eval_freq variables in this file for my less powerful machine

# Original code: 
    # if os.path.exists(models_path + "/" + file_name + str(agent_num) +
    #                   "_policy"):
    #     print("Loading Existing agent")
    #     agent.load(models_path + "/" + file_name + str(agent_num))

# Changed code:
    pretrained = os.path.join(models_path, f"spot_ars{agent_num}_policy")
    if os.path.exists(pretrained):
         print("Loading Existing agent")
        with open(pretrained, 'rb') as f:
            old_theta = pickle.load(f, encoding='latin1') # latin1 encoding helps with older formats
        new_theta = np.zeros((action_dim, state_dim))   # new matrix for the environment dimensions 
                                                        # action_dim is num of output actions, state_dim is the new input size
        new_theta[:, :old_theta.shape[1]] = old_theta   # copy over pretrained weights
        agent.policy.theta = new_theta                  # update agent policies
        agent.policy.state_dim = state_dim
        
    # by adding a ball to the simulation, we add 2 more inputs to the theta (x and y distance to the ball)
    # allows the agent to attempt to learn something new while retaining its old knowledge

In [ ]:
# spot_ars.py line 166

# Original code: 
    # for proc_num in range(num_processes):
    #     p = mp.Process(target=ParallelWorker, args=(childPipes[proc_num], env, state_dim))
    
# Changed code:
    p = mp.Process(target=ParallelWorker, args=(childPipes[proc_num], state_dim))
    
# This is a simple change. When parallel workers were being spawned, the environment was being passed
# to each worker. However, the pybullet was connecting to the physics server first, and then the environment
# being passed wouldn't recognize the physics server when the parallel worker began. More info below

In [ ]:
# ars.py line 59

# Original Code:
    # nb_states = env.observation_space.shape[0]
    # common normalizer
    # normalizer = Normalizer(nb_states)
    # max_action = float(env.action_space.high[0])
    # _ = env.reset()
    # n = 0
    
# Changed Code:
import pybullet as p
from spotmicro.GymEnvs.spot_bezier_env import spotBezierEnv
from spotmicro.spot_env_randomizer import SpotEnvRandomizer

    def ParallelWorker(childPipe, nb_states):
        p.connect(p.DIRECT)    # p.GUI will open up a window, but rendering isn't possible with all the training
        env_randomizer = SpotEnvRandomizer()                # set up environment, environment randomizer was originally included when we had planned
        env = spotBezierEnv(render=False, on_rack=False, height_field=False, draw_foot_path=False, # to include rocky terrain
                            contacts=True, env_randomizer=env_randomizer)
        
        env.seed(0)             # worker seed
        normalizer = Normalizer(nb_states)
        max_action = float(env.action_space.high[0])
        _ = env.reset()
        n = 0
        
# in class Policy() line 237, I changed the num_deltas, num_best_deltas, episode_steps, and 
# exploration_noise variables to smaller numbers so that my laptop would be able to handle it. 
# The numbers I used were 12, 6, 1000, and 0.1 respectively.

In [ ]:
# spot_ars_eval.py line 174

# Original Code: 
    # if os.path.exists(models_path + "/" + file_name + str(agent_num) +
    #                   "_policy"):
    #     print("Loading Existing agent")
    #     agent.load(models_path + "/" + file_name + str(agent_num))
    #     agent.policy.episode_steps = np.inf
    #     policy = agent.policy
    
# Changed code: 
    policy_path = os.path.join(models_path, f"{file_name}{agent_num}_policy")
    if os.path.exists(policy_path):
        print(f"Loading Existing policy from {policy_path}")
        try: 
            with open(policy_path,"rb") as f:
                theta = pickle.load(f, encoding='latin1')
            agent.policy.theta = theta
            agent.policy.episode_steps = np.inf
            policy = agent.policy
            print("Policy loaded successfully")
        except Exception as e:
            print("Failed to load policy: {e}")
    
    # Nothing super new here besides the latin1 encoding and the check to ensure the policy was 
    # loaded correctly while trying to get presaved policies to work
